# Robust overidentification testing in linear IV models

**Author:** Stuart Lane  
**Date:** 03/12/2024

## Overview

This documentation provides an overview of the `oidrobust` for testing overidentifying restrictions in linear IV models. Sections 1 and 2 introduce the necessity of overidentification testing and introduce the model used in this empirical example (see Lane and Windmeijer (2024) for a more general exposition, and also a more detailed account of the empirical example in this document). Section 3 runs through the data and using the package with example code.

In [4]:
x = 2
print(x)

2
